# Support Vector Machines

At this point in the semester, you have learned about support vector machines and kernels and how they are used to find linear and nonlinear decision boundaries for binary classification. In completing this notebook you will become more familiar with some important concepts in machine learning:

- __Mathematical optimization__
    - Review Lagrange multipliers and constrained optimization
- __Kernel methods__
    - Experiment with polynomial and RBF kernels to perform nonlinear classification
    - Observe example scenarios in which linear kernels fail
    - Choose the right kernel for different problems
- __Hyperparameter tuning__
    - Automate the selection of SVM regularization hyperparameter on a held-out dataset
- __Interpreting weights and support vectors__
    - Train a linear SVM for digit detection
    - Interpret weights and support vectors

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import util
import runClassifier
import datasets
import matplotlib.pyplot as plt
import sklearn

import lagrangian
import svm

## Lagrange multipliers and constrained optimization

As you have learned in the lecture on SVMs, Lagrangians help to convert constrained optimization problems to unconstrained ones, which is useful for finding optimal-margin classifiers. Here we will visualize an example of this technique in action.

Let's start with an abstract example. Suppose we have the following constrained optimization problem

$$
\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & x^2+10 \\
& \text{subject to}
& & x \geq 3
\end{aligned}
\end{equation*}
$$

represented by the following figure.

In [ ]:
%matplotlib inline
lagrangian.makePlot('objective')

Applying the Lagrangian method, we'll create a Lagrangian function that we can minimize without constraints. Let's plot the Lagrangian. The plot is 3D, so you can spin it around. You might need to run the cell a few times to get the graphics to show.

In [ ]:
%matplotlib notebook
%matplotlib notebook
lagrangian.makePlot('lagrangian')

The Lagrangian is the surface you see with colors ranging from cool to warm. The axis ranging from -8 to +8 is $x$. The axis ranging from 0 to 10 is $\alpha$. Now, we can maximize this function over $\alpha$ and minimize over $x$. The black line shows the optimal (maximal) value of $\alpha$ for any given $x$. The blue line shows the optimal (minimal) value of $x$ for any given $\alpha$. We can also see this with a contour plot.

In [ ]:
%matplotlib inline
lagrangian.makePlot('contour')

Let's show the optimization problem just as a function of $\alpha$. That is, for any given value of $\alpha$, we find the value of the Lagrangian if we fix $\alpha$ and maximize over $x$.

In [ ]:
%matplotlib inline
lagrangian.makePlot('alpha')

Let's check that this makes sense. We will look at the Lagrangian again.

In [ ]:
%matplotlib notebook
%matplotlib notebook
lagrangian.makePlot('lagrangian')

__QUESTION 1:__ Spin the 3D figure around so that you're looking at it from the perspective of $x$ (so you see -8..8 on the x-axis and the 0..10 axis is going "away" from you). You should be able to see a saddle point in the black curve where it hits a minimum. For what value of $x$ does it attain that minimum? Is this consistent with Figure 4 above (the optimization problem as function of $\alpha$)?

__ANSWER 1:__ 

Suppose we change the constraint to $x \geq -2$, instead of the current $x \geq 3$. Let's make all the plots for this new problem.

In [ ]:
%matplotlib inline
lagrangian.makePlot('all', constraint_x_ge=-2)

In [ ]:
%matplotlib notebook
%matplotlib notebook
lagrangian.makePlot('lagrangian', constraint_x_ge=-2)

__QUESTION 2:__ Repeat question 1, but for this new figure.

__ANSWER 2:__ 

__QUESTION 3:__ Look at the new Figure 4 (the optimization problem as a function of $\alpha$). Where is the optimum for $\alpha$? What does this tell you about the constraint in the constrained optimization?

__ANSWER 3:__ 

## Linear kernel

Now we will experiment with different kernels by applying the SVM to several classification problems. We will use `sklearn.svm`, which is a wrapper around LIBSVM. We will start by training a simple SVM with a linear kernel and regularization parameter C=100. The SVM will be trained on `Data0`, and we will save our model to `models/data0.model`.

In [ ]:
%matplotlib inline
data = datasets.Data0
svm.train(data.X, data.Y,'models/data0.model',k='linear',c=100)
svm.drawBoundary(data.X, data.Y,'models/data0.model')

This is an easily separable dataset, which is reflected by the small number of support vectors. In the plot, the SVs are drawn big (and are on the margin, the dashed line one unit away from the decision boundary, the solid line).

__QUESTION 4:__ You should have found that it takes 3 support vectors. Could you have fewer (eg., 2) support vectors here?

__ANSWER 4:__ 

## Polynomial kernel

Although it is unnecessary, suppose we wanted to use the polynomial kernel $K(x,z)=(1+x\cdot z)^{10}$. Then we would run

In [ ]:
data = datasets.Data0
svm.train(data.X, data.Y,'models/data0.model',k='poly',c=100,r=1,d=10)
svm.drawBoundary(data.X, data.Y,'models/data0.model')

You'll see that you get a curved decision boundary, though of course this is somewhat overkill.

## RBF kernel

Let's try the RBF kernel with $\gamma =1$.

In [ ]:
data = datasets.Data0
svm.train(data.X, data.Y,'models/data0.model',k='rbf',c=100,g=1)
svm.drawBoundary(data.X, data.Y,'models/data0.model')

Again, this is overkill. But we can try to understand RBF kernels a bit better by "turning up" the gamma:

In [ ]:
data = datasets.Data0
svm.train(data.X, data.Y,'models/data0.model',k='rbf',c=100,g=100)
svm.drawBoundary(data.X, data.Y,'models/data0.model')

A gamma of 100 means that you have to be really close to a point to have a kernel value that's non-zero.

__QUESTION 5:__ Why do you get these little blobs?

__ANSWER 5:__

__TASK 1:__ How high do you have to turn gamma up in order to get a little decision boundary around each example? Set gamma below.

In [ ]:
data = datasets.Data0
svm.train(data.X, data.Y,'models/data0.model',k='rbf',c=100,g=100)
svm.drawBoundary(data.X, data.Y,'models/data0.model')

## Choosing the right kernel

Let's now switch to a more complex dataset to see why choosing the right kernel matters. We'll begin by failing with a linear model:

In [ ]:
data = datasets.Data1
svm.train(data.X, data.Y,'models/data1.model',k='linear',c=100)
svm.drawBoundary(data.X, data.Y,'models/data1.model')

As you can see, this data fails horribly.

__QUESTION 6:__ There are a lot of red support vectors on the blue side of the decision boundary. Why?

__ANSWER 6:__ 

However, now we can get some mileage out of polynomial kernels:

In [ ]:
data = datasets.Data1
svm.train(data.X, data.Y,'models/data1.model',k='poly',c=100,r=1,d=3)
svm.drawBoundary(data.X, data.Y,'models/data1.model')

__QUESTION 7:__ Based on this data, is the 0/1 loss on the training data zero? Is the hinge loss on the training data zero?

__ANSWER 7:__

__TASK 2:__ Train an SVM with RBF kernel on this data. What's the smallest gamma for which you can get a good decision boundary?

In [ ]:
# your code here

## SVM regularization parameter

Let's train an SVM on a simple 2-dimensional binary classification task. Our dataset is `BinaryWithOutliers`, and we will save our model to `models/bwo.model`. Experiment with different values of the `C` regularization hyperparameter below.

In [ ]:
%matplotlib inline
data = datasets.BinaryWithOutliers
f = svm.train(data.X, data.Y,'models/bwo.model',k='linear',c=1)
runClassifier.plotDataAndBoundary(data.X,data.Y,f.coef_[0],f.intercept_[0])

__QUESTION 8:__ Which values of `C` lead to underfitting? Which values lead to overfitting? Why?

__ANSWER 8:__ 

This establishes the basic idea of why it is important to tune the regularization parameter. Let's move on to the sentiment prediction task. We want to train a linear SVM on `SentimentData`, and choose the best value for `C`. We will estimate underfitting and overfitting on a held-out subset of the training data. We will use an 80/20 train/validation split of the first 400 examples in `SentimentData`, because training the SVM on the full dataset takes some time. In the real world, you will want to use as much data as possible.

In [ ]:
data = datasets.SentimentData
t = 320
v = 80
trainX,trainY = data.X[0:t],data.Y[0:t]
valX,valY = data.X[t:t+v],data.Y[t:t+v]

__TASK 3:__ Given `trainX`,`trainY`,`valX`,`valY`, write code below to automate the process of choosing a good value for `C`. You must store your best `C` in `bestC`. To compute the score of a trained model `f` on inputs in `X` and corresponding labels in `Y`, write `f.score(X,Y)`.

In [ ]:
bestC = 0 # you must store your best C in this variable
# your code here

In [ ]:
bestC # run this cell so that we can see it

## SVM for digit detection

We will train a support vector machine to classify digits as 1 or 2, with a dataset of 100 grayscale images of resolution 28x28. The images have been reshaped to 784 for training.

In [ ]:
data = datasets.DigitData
f = svm.train(data.X, data.Y,'models/digit.model',k='linear',c=100, prob=True)

It should train very quickly, as the dataset is rather small. Let's get the weight vector and the support vectors. There should be about 28 support vectors.

In [ ]:
w = f.coef_
svs = f.support_vectors_

__TASK 4:__ Write a method `show_wimg` that takes in a weight vector of length 784, converts it to a 28x28 image with color intensity values (0 to 255), and displays it with `plt.imshow`. We will use it to display the weight vector and support vectors.

In [ ]:
def show_wimg(w):
    util.raiseNotDefined() # your code here

Now we can visualize the weight vector.

In [ ]:
show_wimg(w)

__QUESTION 9:__ Inspect the image of the weight vector. What does this weight vector tell you about how it classifies images as either 1 or 2?

__ANSWER 9:__ 

Change the index in the code below to display some support vectors. Try to understand what these support vectors suggest about the classifier. See if you can find anything unique about them.

In [ ]:
show_wimg(svs[0])

__QUESTION 10:__ What did you find? Based on your understanding of how support vectors are used to find an optimal-margin decision boundary, what do the nature of the support vectors, and how many there are, suggest about this SVM's ability to generalize?

__ANSWER 10:__ 

## Learn more

### LIBSVM
- [LIBSVM -- A Library for Support Vector Machines](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)
- [1.4. Support Vector Machines — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/svm.html)